# Call stacks


In [ ]:
# | code-fold: true
# adapted from https://gist.github.com/mkborregaard/81825c3d370bb4d8dbfe59c3b2ae4b33
# by mkborregaard

using JuliaInterpreter, OrderedCollections
using Graphs, MetaGraphsNext

const callchains = OrderedSet{Vector{Method}}()
const modules = Set{Module}()

function callchain(frame::JuliaInterpreter.Frame)
    chain = Method[]
    sc = JuliaInterpreter.scopeof(frame)
    while sc isa Method
        push!(chain, sc)
        frame = frame.caller
        frame === nothing && break
        sc = JuliaInterpreter.scopeof(frame)
    end
    return chain
end

function log_far!(@nospecialize(recurse), frame, istoplevel::Bool=false)
    chain = callchain(frame)
    chain[1].module ∈ modules && push!(callchains, chain)
    return JuliaInterpreter.finish_and_return!(recurse, frame, istoplevel)
end

function encode_vertices(callchains)
    i = 0
    vertices = Dict{Array{Method},Int}()
    for chain in callchains
        for ind in length(chain):-1:1
            vert = chain[ind:end]
            haskey(vertices, vert) || (vertices[vert] = (i += 1))
        end
    end
    vertices
end

function getnames(vertices)
    names = Vector{String}(undef, length(vertices))
    for (k, v) in vertices
        names[v] = "$(k[1].module).$(k[1].name)"
    end
    names
end

function construct_graph(callchains)
    vertices = encode_vertices(callchains)
    names = getnames(vertices)

    graph = MetaGraph(
        DiGraph(); 
        label_type = Int,
        vertex_data_type = String
    )

    for (i, name) in enumerate(names)
        graph[i] = name
    end

    for chain in callchains
        for ind in length(chain)-1:-1:1
            src = vertices[chain[ind+1:end]]
            dst = vertices[chain[ind:end]]
            graph[src, dst] = nothing
        end
    end

    graph
end

function tracecall(mods::Tuple, call, arg)
    empty!(callchains)
    empty!(modules)
    for m in mods
        push!(modules, m)
    end
    frame = JuliaInterpreter.enter_call(call, arg)
    log_far!(log_far!, frame, false)
    construct_graph(callchains)
end

#############################################

using Ribasim

## Model initialization


In [ ]:
toml_path = normpath(@__DIR__, "../../generated_testmodels/trivial/ribasim.toml")
graph = tracecall((Ribasim,), toml_path)

In [ ]:
using GLMakie

f = Figure()

f